In [4]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain, LLMMathChain
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain
from transformers import GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import ShellTool
from langchain.tools import HumanInputRun
from langchain.agents import create_pandas_dataframe_agent
import pandas as pd
from datasets import load_dataset
import torch
import transformers

#change these inport statments to ones i need

In [169]:
import re
from langchain.tools import tool

# Define which tools the agent can use to answer user queries
search = DuckDuckGoSearchRun()

def google_search(input_text):
#   search_results = search.run(f"site:webmd.com {input_text}")
    search_results = search.run(f"site:google.com {input_text}")
    return search_results
@tool
def Multiply(a: int, b: int) -> int:
    """Add two numbers"""
    return a*b

tools = [
    Tool(
        name = "Search Google",
        func = google_search,
        description="useful for getting answers to general questions, or questions that ask you to search google"
    ), 
    Multiply
]

In [171]:
# Set up the base template
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Here are some examples for this format:

Question: What is the product of 4 and 5?
Thought: I should use the Multiply tool
Action: Multiply
Action Input: Multiply(4, 5)
Observation: 20
Thought: I now know the final answer
Final Answer: The product of 4 and 5 is 20

Begin! Remember to use tools and the template when giving your final answer.

Question: {input}
{agent_scratchpad}"""

In [172]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [173]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [174]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "As an AI language model, I cannot" in llm_output:
            raise SyntaxError(f"Output does not follow prompt: {llm_output}")
        
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [175]:
#LLM used is tiiuae/falcon-40b-instruct

llm_chain = LLMChain(llm=__, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]

print(tool_names)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=CustomOutputParser(),
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)


In [177]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, 
                                                    tools=tools, 
                                                    verbose=True)

In [ ]:
langchain.debug = True
while True:
    try:
        agent_executor.run("What is the product of 92 and 43?")
        break
    except Exception as e:
        pass